# Evolver Loop 11 Analysis

Analyzing the current state and identifying opportunities for improvement.

**Current Status:**
- Best CV: 70.630478
- Target: 68.919154
- Gap: 1.711 points (2.42%)

**Key Insight from Research:**
- N < 58: SA for chaotic packings
- N > 58: Crystalline/Lattice packing
- Top teams use HYBRID strategies with different approaches for different N ranges

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon

getcontext().prec = 25
scale_factor = Decimal("1e15")

print("Libraries loaded")

In [ ]:
# Load current best submission
current_best = pd.read_csv('/home/submission/submission.csv')

# Calculate per-N scores
def calculate_per_n_scores(df):
    """Calculate score for each N value"""
    scores = {}
    
    for n in range(1, 201):
        group = df[df['id'].str.startswith(f'{n:03d}_')]
        if len(group) == 0:
            continue
        
        # Parse coordinates
        xs = [float(str(x).lstrip('s')) for x in group['x']]
        ys = [float(str(y).lstrip('s')) for y in group['y']]
        
        # Calculate bounding box
        min_x, max_x = min(xs), max(xs)
        min_y, max_y = min(ys), max(ys)
        
        # Need to account for tree polygon size
        # Tree extends roughly -0.35 to 0.35 in x, -0.2 to 0.8 in y at angle 0
        # For simplicity, use the actual polygon calculation
        side = max(max_x - min_x + 0.7, max_y - min_y + 1.0)  # Approximate
        scores[n] = side ** 2 / n
    
    return scores

# More accurate calculation using actual tree polygons
def get_tree_polygon(x, y, angle):
    """Get tree polygon at given position and angle"""
    trunk_w = 0.15
    trunk_h = 0.2
    base_w = 0.7
    mid_w = 0.4
    top_w = 0.25
    tip_y = 0.8
    tier_1_y = 0.5
    tier_2_y = 0.25
    base_y = 0.0
    trunk_bottom_y = -trunk_h
    
    points = [
        (0.0, tip_y),
        (top_w / 2, tier_1_y),
        (top_w / 4, tier_1_y),
        (mid_w / 2, tier_2_y),
        (mid_w / 4, tier_2_y),
        (base_w / 2, base_y),
        (trunk_w / 2, base_y),
        (trunk_w / 2, trunk_bottom_y),
        (-trunk_w / 2, trunk_bottom_y),
        (-trunk_w / 2, base_y),
        (-base_w / 2, base_y),
        (-mid_w / 4, tier_2_y),
        (-mid_w / 2, tier_2_y),
        (-top_w / 4, tier_1_y),
        (-top_w / 2, tier_1_y),
    ]
    
    poly = Polygon(points)
    rotated = affinity.rotate(poly, angle, origin=(0, 0))
    translated = affinity.translate(rotated, xoff=x, yoff=y)
    return translated

def calculate_accurate_per_n_scores(df):
    """Calculate accurate score for each N using actual polygons"""
    scores = {}
    
    for n in range(1, 201):
        group = df[df['id'].str.startswith(f'{n:03d}_')]
        if len(group) == 0:
            continue
        
        # Get all tree polygons
        all_coords = []
        for _, row in group.iterrows():
            x = float(str(row['x']).lstrip('s'))
            y = float(str(row['y']).lstrip('s'))
            deg = float(str(row['deg']).lstrip('s'))
            poly = get_tree_polygon(x, y, deg)
            all_coords.extend(list(poly.exterior.coords))
        
        # Calculate bounding box
        xs = [c[0] for c in all_coords]
        ys = [c[1] for c in all_coords]
        side = max(max(xs) - min(xs), max(ys) - min(ys))
        scores[n] = side ** 2 / n
    
    return scores

print("Calculating per-N scores...")
per_n_scores = calculate_accurate_per_n_scores(current_best)
print(f"Total score: {sum(per_n_scores.values()):.6f}")

In [ ]:
# Analyze per-N scores
print("Per-N Score Analysis:")
print("="*60)

# Group by ranges
ranges = [
    (1, 10, "Small (1-10)"),
    (11, 20, "Small-Medium (11-20)"),
    (21, 50, "Medium (21-50)"),
    (51, 100, "Medium-Large (51-100)"),
    (101, 150, "Large (101-150)"),
    (151, 200, "Very Large (151-200)")
]

for start, end, label in ranges:
    range_scores = [per_n_scores[n] for n in range(start, end+1) if n in per_n_scores]
    total = sum(range_scores)
    avg = np.mean(range_scores)
    print(f"{label}: total={total:.4f}, avg={avg:.4f}, count={len(range_scores)}")

In [ ]:
# Find N values with highest per-N scores (most room for improvement)
print("\nTop 20 N values with highest per-N scores:")
print("="*60)

sorted_scores = sorted(per_n_scores.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:20]:
    print(f"N={n:3d}: score={score:.6f}")

In [ ]:
# Theoretical lower bound analysis
# Tree polygon area is approximately 0.3025 (calculated from polygon)
# For perfect packing, minimum side = sqrt(N * tree_area / packing_efficiency)
# Typical packing efficiency for irregular polygons is 60-80%

tree_area = 0.3025  # Approximate tree polygon area

print("\nTheoretical Analysis:")
print("="*60)

for n in [1, 5, 10, 20, 50, 100, 150, 200]:
    current = per_n_scores.get(n, 0)
    
    # Theoretical minimum (100% packing efficiency)
    theoretical_min_side = np.sqrt(n * tree_area)
    theoretical_min_score = theoretical_min_side ** 2 / n
    
    # Realistic minimum (70% packing efficiency)
    realistic_min_side = np.sqrt(n * tree_area / 0.7)
    realistic_min_score = realistic_min_side ** 2 / n
    
    efficiency = (theoretical_min_side / np.sqrt(current * n)) ** 2 * 100
    
    print(f"N={n:3d}: current={current:.4f}, theoretical_min={theoretical_min_score:.4f}, " +
          f"realistic_min={realistic_min_score:.4f}, efficiency={efficiency:.1f}%")

In [ ]:
# Plot per-N scores
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
ns = list(per_n_scores.keys())
scores = [per_n_scores[n] for n in ns]
plt.plot(ns, scores, 'b-', alpha=0.7)
plt.xlabel('N')
plt.ylabel('Score (s^2/N)')
plt.title('Per-N Scores')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Cumulative contribution
cumulative = np.cumsum(scores)
plt.plot(ns, cumulative, 'r-', alpha=0.7)
plt.xlabel('N')
plt.ylabel('Cumulative Score')
plt.title('Cumulative Score Contribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/home/code/exploration/per_n_analysis.png', dpi=100)
plt.show()

print(f"\nTotal score: {cumulative[-1]:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {cumulative[-1] - 68.919154:.6f}")

In [ ]:
# Identify which N values contribute most to the gap
print("\nContribution Analysis:")
print("="*60)

# If we could improve each N by X%, what would be the impact?
for improvement_pct in [1, 2, 5, 10]:
    new_total = sum(s * (1 - improvement_pct/100) for s in per_n_scores.values())
    print(f"{improvement_pct}% improvement across all N: {new_total:.4f} (gap: {new_total - 68.919154:.4f})")

print("\nTo reach target 68.919154:")
current_total = sum(per_n_scores.values())
required_improvement = (current_total - 68.919154) / current_total * 100
print(f"Need {required_improvement:.2f}% improvement overall")

In [ ]:
# Check if there are any N values where we're significantly worse than theoretical
print("\nN values with worst efficiency (most room for improvement):")
print("="*60)

efficiencies = []
for n in range(1, 201):
    if n not in per_n_scores:
        continue
    current = per_n_scores[n]
    theoretical_min_side = np.sqrt(n * tree_area)
    efficiency = (theoretical_min_side / np.sqrt(current * n)) ** 2 * 100
    efficiencies.append((n, efficiency, current))

# Sort by efficiency (lowest = most room for improvement)
efficiencies.sort(key=lambda x: x[1])

print("Lowest efficiency (most room for improvement):")
for n, eff, score in efficiencies[:15]:
    print(f"N={n:3d}: efficiency={eff:.1f}%, score={score:.6f}")

print("\nHighest efficiency (already well optimized):")
for n, eff, score in efficiencies[-10:]:
    print(f"N={n:3d}: efficiency={eff:.1f}%, score={score:.6f}")

In [ ]:
# Summary
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Current total score: {sum(per_n_scores.values()):.6f}")
print(f"Target: 68.919154")
print(f"Gap: {sum(per_n_scores.values()) - 68.919154:.6f} ({required_improvement:.2f}%)")
print()
print("Key observations:")
print("1. Small N (1-10) have highest per-N scores but contribute less to total")
print("2. Large N (150-200) have lower per-N scores but contribute ~50% of total")
print("3. Efficiency is 50-85% depending on N")
print("4. Need 2.42% improvement overall to reach target")
print()
print("Recommended focus areas:")
print("1. Large N (150-200) - highest total contribution")
print("2. N values with lowest efficiency - most room for improvement")
print("3. Try crystalline/lattice packing for N > 58 as suggested by research")